### DB 연결

In [1]:
import psycopg2

conn = psycopg2.connect(
  'host=localhost port=5432 dbname=postgres user=postgres password=postgres',
  options='-c search_path=sbd'
)
conn.set_session(autocommit=True)
cur = conn.cursor()

### 테이블 생성

In [2]:
cur.execute(
  f'''
  select count(*)
  from information_schema.tables
  where
    table_schema = 'sbd' and
    table_name ~ 'building_floor'
  '''
)
if not cur.fetchone()[0]:
  cur.execute(
    open('sql/source-create_table_building_floor.sql', 'r').read()
  )

후속 작업에 필요한 컬럼 목록, 테이블 초기화

In [3]:
cur.execute(
  f'''
  select column_name
  from information_schema.columns
  where 
    table_schema = 'sbd' and
    table_name = 'building_floor' and
    column_default is null
  '''
)
column_list = [
  col[0]
  for col
  in cur.fetchall()
]

In [4]:
cur.execute(
  'delete from building_floor'
)

### 업로드 함수

In [5]:
def uploadToDB(data_table):
  print(datetime.now(),'start...',end='')
  data_table.columns = [
    '관리_건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명', '층_구분_코드', '층_구분_코드_명', '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '면적(㎡)', '주_부속_구분_코드', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자'
  ]
  data_table['시도_코드'] = [code[0:2] for code in data_table['시군구_코드']]
  data_table = data_table[data_table['시도_코드'] == '11'] # '전국' 중 '서울'만 추출
  data_table['pnu'] = data_table['시군구_코드'] + data_table['법정동_코드'] + [ # 토지 ID인 PNU 생성
    '1' if code == '0' else '2' if code == '1' else '0'
    for code
    in data_table['대지_구분_코드']
  ] + data_table['번'] + data_table['지']
  data_table = data_table[[
    'pnu', '관리_건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '동_명', '층_구분_코드', '층_구분_코드_명', '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '면적(㎡)', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자'
  ]]
  data_table.columns = column_list
  data_table.floor_no_name = data_table.floor_no_name.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.structure_detail = data_table.structure_detail.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.use_detail = data_table.use_detail.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.is_excepted_area = [ # 0,1 -> False, True
    False if (pd.isna(tf) or tf == '0') else True
    for tf
    in data_table['is_excepted_area']
  ]
  data_table.to_csv( # .txt 파일로 로컬 저장
    'temp_building_floor.txt',
    sep='|',
    index=False,
    header=False,
    encoding='CP949'
  )
  temp_file = open('temp_building_floor.txt', 'r')
  print('upload...',end='')
  cur.copy_from( # file bulk insert
    temp_file,
    'building_floor',
    sep='|',
    columns=column_list,
    null=''
  )
  temp_file.close()
  os.remove('temp_building_floor.txt')
  print('end',datetime.now())

### 소스 데이터 업로드

In [6]:
from zipfile import ZipFile
import pandas as pd
import os
from datetime import datetime

zf = ZipFile('D:/data/building_register/building_floor/국토교통부_건축물대장_층별개요+(2022년+07월).zip')
source_chunks = pd.read_csv(
  zf.open('mart_djy_04.txt'),
  sep='|',
  encoding='CP949',
  header=None,
  dtype='string',
  chunksize=500_000
)

In [7]:
for source_dt in source_chunks:
  uploadToDB(source_dt)

2022-09-23 17:32:32.576618 start...

C:\Users\Bob\AppData\Local\Temp/ipykernel_14524/2940040397.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table['pnu'] = data_table['시군구_코드'] + data_table['법정동_코드'] + [ # 토지 ID인 PNU 생성


upload...end 2022-09-23 17:32:35.737760
2022-09-23 17:32:40.306091 start...upload...end 2022-09-23 17:32:43.407229
2022-09-23 17:32:47.894397 start...upload...end 2022-09-23 17:32:51.026488
2022-09-23 17:32:55.658996 start...upload...end 2022-09-23 17:32:58.862616
2022-09-23 17:33:03.521509 start...upload...end 2022-09-23 17:33:06.749325
2022-09-23 17:33:11.252995 start...upload...end 2022-09-23 17:33:14.467592
2022-09-23 17:33:18.912020 start...upload...end 2022-09-23 17:33:22.117162
2022-09-23 17:33:26.598841 start...upload...end 2022-09-23 17:33:29.821926
2022-09-23 17:33:34.556433 start...upload...end 2022-09-23 17:33:37.876935
2022-09-23 17:33:42.729026 start...upload...end 2022-09-23 17:33:46.228090
2022-09-23 17:33:50.894020 start...upload...end 2022-09-23 17:33:54.430410
2022-09-23 17:33:58.913327 start...upload...end 2022-09-23 17:34:02.653941
2022-09-23 17:34:07.153225 start...upload...end 2022-09-23 17:34:10.592416
2022-09-23 17:34:15.213725 start...upload...end 2022-09-23 1